In [ ]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pavia_data = sio.loadmat('PaviaU.mat')
pavia_gt = sio.loadmat('PaviaU_gt.mat')

pavia_image = pavia_data['paviaU']
pavia_labels = pavia_gt['paviaU_gt']

height, width, bands = pavia_image.shape
print("Dataset : Pavia University")
print(f"Dimensions de l’image : {height} x {width}")
print(f"Nombre de bandes spectrales : {bands}")
print(f"Dimensions des étiquettes : {pavia_labels.shape}")

print("\nDescription :")
print("L’image hyperspectrale est un cube 3D : (hauteur x largeur x bandes spectrales).")
print("Chaque pixel contient 103 valeurs, chacune correspondant à une longueur d’onde différente.")
print("Cela permet d’identifier des matériaux non visibles par l’œil humain.")

num_labeled = np.count_nonzero(pavia_labels)
print(f"\nPixels étiquetés : {num_labeled}")
print(f"Pixels non étiquetés : {(height * width) - num_labeled}")

unique_classes, counts = np.unique(pavia_labels, return_counts=True)
print("\nRépartition des classes :")
for cls, cnt in zip(unique_classes, counts):
    print(f"  Classe {cls} : {cnt} pixels ")

print("\nPremières lignes de l’image (bande 0) :")
print(pavia_image[:20, :, 0])

print("\nPremières lignes des étiquettes :")
print(pavia_labels[:20, :])

plt.figure(figsize=(8, 4))
sns.barplot(x=unique_classes, y=counts)
plt.title("Répartition des classes")
plt.xlabel("Classe")
plt.ylabel("Nombre de pixels")
plt.show()

def normalize_image(img):
    img = img.astype(np.float32)
    img -= img.min()
    img /= img.max()
    return img

rgb_image = pavia_image[:, :, [55, 36, 10]]
rgb_image = normalize_image(rgb_image)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(rgb_image)
plt.title("Image RGB (bandes 30, 20, 10)")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(pavia_labels, cmap='nipy_spectral')
plt.title("Vérité terrain")
plt.axis('off')

plt.tight_layout()
plt.show()

print("\nObservations visuelles :")
print("- Les zones rectangulaires correspondent à des bâtiments ou toits (classe 5).")
print("- Les bandes sombres et linéaires indiquent les routes ou le bitume (classes 1 et 7).")
print("- Les zones texturées représentent la végétation (classes 2 et 4).")
print("- Les taches très sombres indiquent des ombres (classe 9).")
print("- Certaines classes sont spectralesment proches, rendant la classification difficile.")


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

print("Mise en forme des données pour la PCA")
X = pavia_image.reshape(-1, bands)
print(f"Données aplaties : {X.shape[0]} pixels, {X.shape[1]} bandes")

print("Standardisation (centrage/réduction)...")
X_std = StandardScaler().fit_transform(X)

pca_all = PCA(n_components=bands)
X_pca_all = pca_all.fit_transform(X_std)

plt.figure(figsize=(8, 6))
plt.plot(range(1, bands+1), np.cumsum(pca_all.explained_variance_ratio_), marker='o', linestyle='--', color='b')
plt.title("Variance expliquée par les composantes principales")
plt.xlabel("Nombre de composantes")
plt.ylabel("Variance cumulée (%)")
plt.axhline(y=0.95, color='r', linestyle='--', label="95% de variance")
plt.legend()
plt.grid(True)
plt.show()

for i, var in enumerate(np.cumsum(pca_all.explained_variance_ratio_)):
    if var >= 0.95:
        print(f"Pour expliquer 95% de la variance, il faut {i+1} composantes.")
        break

print("Application de la PCA (n_components=3)...")
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_std)

print("Variance expliquée par les 3 premières composantes :")
for i, var in enumerate(pca.explained_variance_ratio_[:3]):
    print(f"  Composante {i+1} : {var*100:.2f}%")

total_var = np.sum(pca.explained_variance_ratio_[:3])
print(f"Total expliqué par les 3 premières composantes : {total_var*100:.2f}%")

pca_image = X_pca.reshape(height, width, 3)
pca_image = normalize_image(pca_image)

plt.figure(figsize=(6, 5))
plt.imshow(pca_image)
plt.title("Image reconstruite à partir des 3 premières composantes principales")
plt.axis('off')
plt.show()



In [ ]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ip_data = sio.loadmat('Indian_pines.mat')
ip_gt = sio.loadmat('Indian_pines_gt.mat')

ip_image = ip_data['indian_pines']
ip_labels = ip_gt['indian_pines_gt']

height, width, bands = ip_image.shape
print("Dataset : Indian Pines")
print(f"Dimensions de l'image : {height} x {width}")
print(f"Nombre de bandes spectrales : {bands}")
print(f"Taille du masque de vérité terrain : {ip_labels.shape}")

num_labeled = np.count_nonzero(ip_labels)
print(f"Pixels étiquetés : {num_labeled}")
print(f"Pixels non étiquetés (0) : {(height * width) - num_labeled}")

unique_classes, counts = np.unique(ip_labels, return_counts=True)
print("Répartition des classes :")
for cls, cnt in zip(unique_classes, counts):
    print(f"  Classe {cls} : {cnt} pixels")

plt.figure(figsize=(8, 4))
sns.barplot(x=unique_classes, y=counts)
plt.title("Répartition des classes dans Indian Pines")
plt.xlabel("Étiquette de classe")
plt.ylabel("Nombre de pixels")
plt.show()

plt.figure(figsize=(6, 5))
plt.imshow(ip_labels, cmap='nipy_spectral')
plt.title("Vérité terrain - Indian Pines")
plt.axis('off')
plt.show()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = ip_image.reshape(-1, bands)
print(f"Données aplaties : {X.shape[0]} pixels, {X.shape[1]} bandes")

print("Standardisation (centrage/réduction)...")
X_std = StandardScaler().fit_transform(X)

pca_all = PCA(n_components=bands)
X_pca_all = pca_all.fit_transform(X_std)

plt.figure(figsize=(8, 6))
plt.plot(range(1, bands+1), np.cumsum(pca_all.explained_variance_ratio_), marker='o', linestyle='--', color='b')
plt.title("Variance expliquée par les composantes principales")
plt.xlabel("Nombre de composantes")
plt.ylabel("Variance cumulée (%)")
plt.axhline(y=0.95, color='r', linestyle='--', label="95% de variance")
plt.legend()
plt.grid(True)
plt.show()

for i, var in enumerate(np.cumsum(pca_all.explained_variance_ratio_)):
    if var >= 0.95:
        print(f"Pour expliquer 95% de la variance, il faut {i+1} composantes.")
        break

print("Application de la PCA (n_components=27)...")
pca = PCA(n_components=27) 
X_pca = pca.fit_transform(X_std)

print("Variance expliquée par les 9 premières composantes :")
for i, var in enumerate(pca.explained_variance_ratio_[:27]):
    print(f"  Composante {i+1} : {var*100:.2f}%")

total_var = np.sum(pca.explained_variance_ratio_[:27])
print(f"Total expliqué par les 27 premières composantes : {total_var*100:.2f}%")

def normalize_image(img):
    img = img.astype(np.float32)
    img -= img.min()
    img /= img.max()
    return img

pca_image = X_pca[:, :3].reshape(height, width, 3)  
pca_image = normalize_image(pca_image)

plt.figure(figsize=(6, 5))
plt.imshow(pca_image)
plt.title("Image PCA (3 composantes) - Indian Pines")
plt.axis('off')
plt.show()

print("La PCA a réduit les 220 bandes à 27 composantes principales.")
print("On visualise plus facilement les structures spatiales.")
print("Cela permet une analyse plus rapide ou une préparation à la classification.")
